In [ ]:
# Disables Weights & Biases logging so the Trainer does not ask for an API key (via Github)
import os
os.environ["WANDB_DISABLED"] = "true"




In [ ]:
!pip install planetterp transformers torch pandas scikit-learn

import planetterp
import pandas as pd
import torch


In [ ]:
# Retrieves professor reviews from PlanetTerp using correct full names
professors = [
    "Nelson Padua-Perez",
    "Timothy Pilachowski",
    "Pedram Sadeghian",
    "Stefan Doboszczak",
    "Jonathan Fernandes"
]

all_reviews = []

for prof in professors:
    print(f"Fetching: {prof}")
    data = planetterp.professor(prof, reviews=True)

    # If PlanetTerp returns any  error, this skips
    if not isinstance(data, dict) or "reviews" not in data:
        print(" -> ERROR: No reviews found")
        continue

    # Extracts each review
    for r in data["reviews"]:
        all_reviews.append({
            "professor": prof,
            "review_text": r.get("review", ""),
            "rating": r.get("rating", None)
        })

print(f"Collected {len(all_reviews)} total reviews.")

df = pd.DataFrame(all_reviews)
df.head()

In [ ]:
# Removes blank reviews and convert ratings to integers
df = df.dropna(subset=["review_text", "rating"])
df = df[df["review_text"].str.strip() != ""]
df["rating"] = df["rating"].astype(int)
df.head()


In [ ]:
# Loads tokenizer and convert all review text to tokenized model inputs
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

texts = df["review_text"].tolist()
labels = df["rating"].tolist()

encodings = tokenizer(
    texts,
    truncation=True,
    padding=True
)


In [ ]:
# PyTorch Dataset for tokenized reviews and labels
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx] - 1)  # convert 1–5 to 0–4
        return item

dataset = ReviewDataset(encodings, labels)


In [ ]:
# Loads the DistilBERT model set up to predict ratings from 1 to 5
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=5
)


In [ ]:
# Defines how the model trains and creates the Trainer that runs fine-tuning
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    logging_steps=20,
    weight_decay=0.01,
    save_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)


In [ ]:
# Trains the DistilBERT model on the collected review dataset
trainer.train()

In [ ]:
# prediction helper that generates the model’s star ratings
def predict_rating(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return pred + 1

df["predicted"] = df["review_text"].apply(predict_rating)
df.head()


In [ ]:
# Evaluates the model’s performance using accuracy and mean absolute error
from sklearn.metrics import accuracy_score, mean_absolute_error

accuracy = accuracy_score(df["rating"], df["predicted"])
mae = mean_absolute_error(df["rating"], df["predicted"])

print("Accuracy:", accuracy)
print("MAE:", mae)


In [ ]:

# Visualization that hows how predicted ratings compare to actual ratings
# using a confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(df["rating"], df["predicted"])

sns.heatmap(cm, annot=True, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
